## imports

In [1]:
import os
import socket
import sys

import pandas as pd

from sqlalchemy import create_engine, text


fn_res = r'..\from_sql\df.parquet'

# кортеж с именами необходимых для работы программы служебных файлов
tup_fns = (
    'user_functions.py',
    'paths_and_constants.py',
    'dict_sql_queries.py',
    'get_dicts_v_06.py',
)


# функция, которая проверяет отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
def check_missing_files(tup_fns, path_):
    # Получаем список файлов в указанной папке
    files_in_directory = os.listdir(path_)
    
    # Создаём список файлов, которых нет в папке
    missing_files = [file for file in tup_fns if file not in files_in_directory]
    
    # Проверка и вывод результата
    if missing_files:
        print(f"Следующие файлы отсутствуют в папке '{path_}': {', '.join(missing_files)}")
        sys.exit()
    else:
        print(f"Все необходимые для работы программы служебные файлы присутствуют в папке '{path_}'.")
    
    return missing_files

# получаем имя компьютера
hostname = socket.gethostname()

# если программа выполняется на сервере
if hostname == '304-007':
    # указываем путь к общим документам
    path_ = r'C:\Users\Public\dictionaries'
# иначе (если программа выполняется на своём рабочем ПК)
else:
    # указываем путь к документам активного пользователя
    path_ = os.path.expanduser('~\\Documents\\')

# проверяем отсутствие имен файлов из кортежа tup_fns в содержимом папки path_
check_missing_files(tup_fns, path_)

# добавляем путь к системному пути поиска модулей (путь к папке, в которой Python ищет модули)
sys.path.append(path_)

# далее импорт пользовательских библиотек идёт с пути `path_`
try:
    from user_functions import create_sql_connection, get_sql_user_info_from_txt_file
    from paths_and_constants import cinfo, tup_server_1_big
except:
    print(f'\nДоп. модули, необходиимые для работы кода, НЕ НАЙДЕНЫ по адресу "{path_}"')


Все необходимые для работы программы служебные файлы присутствуют в папке 'C:\Users\VAIO\Documents\'.

Начинаю загружать пути

hostname = "304-004"

Файл с авторизационными данными пользователя (роль) "sql_user_info.txt" найден по адресу "C:\Users\VAIO\Documents\sql_user_info.txt"

hostname = "304-004"

Файл с авторизационными данными пользователя (роль) "sql_user_info.txt" найден по адресу "C:\Users\VAIO\Documents\sql_user_info.txt"

Начинаю загружать доп. переменные и словари


## constants

In [2]:
schema = 'big_data'
tablename = 'df_5213'

## create sql connection

In [3]:
[user, pwd] = get_sql_user_info_from_txt_file(cinfo)

engine = create_sql_connection(user, pwd, tup_server_1_big)

## upload table from sql

In [4]:
user_answer = input('Do You want to read the entire table or its part usinq Your query? Press "E" for entire or another button for the query: ')

### entire table

In [5]:
if user_answer.lower() == 'e':
    with engine.connect() as conn:
        # df_5213 = pd.read_sql_table('df_5213', conn, schema, )
        exec(f"{tablename} = pd.read_sql_table('{tablename}', conn, schema, )")
        

### using a query

In [6]:
if user_answer.lower() != 'e':
    query = '''
    WITH unnested_dj AS
    (
        SELECT 
            pat_id, 
            recipe_info
        FROM 
            recipes.temp_7837_2024_09_23, 
            UNNEST(regexp_split_to_array(dj_rec_info, '&')) AS recipe_info
        WHERE TRUE
            AND LEFT (split_part(recipe_info, '--', 2), 7) >= '2024.09'
            AND LEFT (split_part(recipe_info, '--', 2), 7) < '2024.10'
    )
    , unnested_fo AS
    (
        SELECT 
            pat_id, 
            recipe_info
        FROM 
            recipes.temp_7837_2024_09_23, 
            UNNEST(regexp_split_to_array(fo_rec_info, '&')) AS recipe_info
        WHERE TRUE
            AND LEFT (split_part(recipe_info, '--', 2), 7) >= '2024.09'
            AND LEFT (split_part(recipe_info, '--', 2), 7) < '2024.10'
    )
    , unnested_ju AS
    (
        SELECT 
            pat_id, 
            recipe_info
        FROM 
            recipes.temp_7837_2024_09_23, 
            UNNEST(regexp_split_to_array(ju_rec_info, '&')) AS recipe_info
        WHERE TRUE
            AND ju_rec_info NOTNULL
            AND LEFT (split_part(recipe_info, '--', 2), 7) >= '2024.09'
            AND LEFT (split_part(recipe_info, '--', 2), 7) < '2024.10'
    )
SELECT 
    pat_id
    , 'dj' AS "drug"
    , LEFT (split_part(recipe_info, '--', 2), 10) AS date
FROM 
    unnested_dj
UNION ALL
SELECT
    pat_id
    , 'fo' AS "drug"
    , LEFT (split_part(recipe_info, '--', 2), 10) AS date
FROM 
    unnested_fo
UNION ALL
SELECT 
    pat_id
    , 'ju' AS "drug"
    , LEFT (split_part(recipe_info, '--', 2), 10) AS date
FROM 
    unnested_ju
;
    '''

    # query = query.format(
    #     schema=schema,
    #     tablename=tablename,
    # )

    with engine.connect() as conn:
        df = pd.read_sql_query(query, conn, )
        

In [ ]:
df

In [10]:
if not os.path.exists(fn_res.rsplit(rf'{os.sep}', 1)[0]):
    os.makedirs(fn_res.rsplit(rf'{os.sep}', 1)[0])

df.to_parquet(fn_res)

In [ ]:
os.startfile(fn_res.rsplit(rf'{os.sep}', 1)[0])